<a href="https://colab.research.google.com/github/tylaar1/Quantum-GANs/blob/main/FID_tutorial_numpyVSkeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [37]:
import numpy as np
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize

# Numpy implementation

### function to calculate FID

In [81]:
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

### example of calculating the FID

In [82]:
act1 = fromcode1
act2 = fromcode2

In [83]:
act1.shape

(10, 64)

In [84]:
# act1 = np.random.random(10*2048) # generates 20,480 random numbers
# act1 = act1.reshape((10,2048)) # reshape so you have 10 images worth of feature vectors of length 2048

# act2 = np.random.random(10*2048)
# act2 = act2.reshape((10,2048))

In [85]:
# fid between act1 and act1
fid = calculate_fid(act1, act1)
print('FID (same): %.3f' % fid)

FID (same): -0.000


In [86]:
# fid between act1 and act2
fid = calculate_fid(act1, act2)
print('FID (different): %.3f' % fid)

FID (different): 1.941


# Keras implementation

In [46]:
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = np.resize(image, new_shape)
		# store
		images_list.append(new_image)
	return np.asarray(images_list)

In [51]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [71]:
model = InceptionV3(include_top=False, pooling='avg')

In [72]:
fromcode1 = np.array([[1.0000e+00, 1.8105e-01, 8.7915e-02, 1.3163e-01, 3.0051e-01, 3.3772e-02,
         1.8352e-02, 1.1066e-01, 6.5743e-02, 1.1680e-01, 2.5119e-01, 3.8249e-02,
         3.3713e-02, 2.7016e-02, 5.3321e-01, 5.7228e-03, 7.0915e-02, 5.7505e-02,
         9.7763e-02, 3.4474e-02, 2.6129e-01, 4.1998e-02, 6.1446e-02, 4.9416e-03,
         6.8350e-02, 7.6981e-03, 8.4146e-01, 5.0592e-02, 5.5784e-01, 2.9357e-02,
         1.0000e+00, 1.5608e-02, 1.0000e+00, 7.2338e-03, 1.7637e-01, 1.5435e-01,
         1.4347e-01, 1.3301e-02, 2.2388e-01, 8.3369e-03, 9.7671e-02, 1.5737e-02,
         5.3029e-02, 4.8857e-02, 2.4605e-01, 2.0970e-02, 7.7632e-01, 1.4362e-01,
         1.1210e-01, 1.6222e-02, 4.2412e-01, 2.5998e-04, 7.5380e-02, 3.2457e-05,
         3.5778e-02, 2.1237e-03, 1.3116e-03, 7.4614e-02, 3.9598e-01, 5.9667e-02,
         4.5350e-02, 2.9907e-01, 1.0000e+00, 6.9835e-02],
        [1.0000e+00, 5.7859e-05, 1.5541e-02, 2.4033e-04, 1.0200e-01, 2.0832e-05,
         5.5117e-02, 7.2385e-04, 5.0736e-02, 2.1848e-03, 2.3657e-02, 1.7289e-03,
         1.1922e-02, 1.0660e-04, 2.3937e-01, 2.2598e-02, 2.3401e-01, 1.9959e-01,
         5.3767e-02, 1.4167e-02, 2.1373e-01, 7.2721e-02, 6.5922e-03, 8.2059e-02,
         5.0502e-01, 1.0880e-02, 2.6335e-01, 1.9506e-03, 3.6832e-01, 3.9893e-02,
         1.0000e+00, 5.7264e-02, 1.0000e+00, 7.9750e-02, 1.3258e-01, 2.7315e-01,
         8.8529e-02, 3.7220e-02, 1.0850e-01, 3.0182e-02, 8.9597e-02, 5.6595e-03,
         6.9838e-04, 1.4713e-02, 1.6002e-01, 1.3060e-01, 5.1889e-01, 1.2368e-01,
         1.4902e-01, 4.8392e-03, 3.6835e-01, 2.3108e-02, 1.0239e-01, 1.9143e-02,
         1.9863e-02, 2.5689e-04, 1.4216e-01, 6.9779e-02, 2.8545e-02, 4.4257e-02,
         3.3823e-02, 5.7234e-02, 1.0000e+00, 1.7953e-01],
        [1.0000e+00, 1.5055e-01, 5.1223e-02, 7.6611e-02, 2.7545e-01, 4.6648e-02,
         5.6993e-02, 5.6372e-02, 2.7318e-02, 6.8085e-02, 2.7950e-01, 1.0339e-02,
         3.7165e-02, 2.4221e-02, 7.1970e-01, 3.7487e-04, 5.5238e-02, 1.1475e-01,
         5.7981e-02, 7.8269e-02, 2.7492e-01, 9.0949e-02, 6.9646e-02, 1.8672e-02,
         9.6322e-02, 9.7677e-03, 9.0939e-01, 1.1254e-01, 7.2993e-01, 2.9406e-02,
         1.0000e+00, 4.3027e-02, 9.7990e-01, 4.5172e-04, 1.4777e-01, 2.0176e-01,
         1.8533e-01, 2.5674e-02, 2.5620e-01, 2.4033e-02, 7.3308e-02, 1.2728e-02,
         5.5050e-02, 6.4275e-02, 3.5673e-01, 5.7951e-02, 1.0000e+00, 2.6579e-01,
         8.7788e-02, 7.4660e-03, 2.8206e-01, 2.2861e-03, 6.9194e-02, 1.6888e-04,
         3.0818e-02, 3.8975e-04, 2.1177e-03, 7.5098e-02, 4.5002e-01, 1.1464e-01,
         5.1516e-02, 3.2820e-01, 1.0000e+00, 1.3891e-01],
        [1.0000e+00, 9.7468e-02, 1.4414e-02, 1.0136e-03, 1.6652e-01, 1.2859e-02,
         7.2854e-02, 2.0275e-03, 9.5418e-03, 2.0804e-03, 6.6979e-02, 2.3277e-04,
         2.5151e-02, 4.7269e-03, 2.5058e-01, 7.5677e-04, 4.4953e-01, 4.2636e-01,
         1.3910e-01, 1.2760e-01, 5.9496e-01, 4.1268e-01, 4.8391e-03, 1.6627e-02,
         3.8615e-01, 1.8424e-01, 5.2910e-01, 1.3622e-01, 5.5605e-01, 1.1176e-02,
         1.0000e+00, 2.4869e-02, 1.0000e+00, 4.6101e-01, 8.6752e-02, 8.3537e-02,
         2.1109e-01, 1.6777e-01, 2.2862e-01, 1.8955e-02, 1.2379e-02, 5.0425e-02,
         1.2007e-02, 2.3108e-02, 3.5446e-01, 2.5226e-01, 8.6382e-01, 3.3555e-01,
         1.3670e-01, 7.2009e-02, 2.4588e-01, 7.2976e-03, 1.2075e-01, 6.2317e-02,
         5.2999e-02, 1.7288e-03, 1.3182e-01, 2.8972e-01, 1.1640e-01, 2.3078e-01,
         3.2210e-02, 2.2345e-01, 1.0000e+00, 3.8671e-01],
        [1.0000e+00, 6.5032e-04, 5.2497e-02, 7.5460e-02, 6.7677e-02, 1.6901e-02,
         1.7456e-01, 1.4084e-03, 8.6696e-02, 4.2941e-02, 5.4329e-02, 2.7136e-04,
         3.4297e-02, 1.9340e-02, 7.3594e-01, 1.6076e-01, 5.0794e-02, 5.0595e-02,
         3.1348e-02, 1.2748e-02, 7.2412e-02, 1.3695e-02, 9.6186e-03, 1.6993e-02,
         1.9041e-01, 1.3613e-02, 3.8649e-01, 2.0938e-04, 2.7813e-01, 4.8306e-04,
         1.0000e+00, 3.6330e-03, 6.9466e-01, 4.9989e-04, 1.3576e-01, 6.4143e-02,
         8.7065e-02, 3.1735e-03, 1.8233e-01, 2.1954e-02, 1.0229e-01, 6.6973e-03,
         1.1701e-03, 1.2236e-02, 2.3214e-01, 4.0303e-02, 1.0000e+00, 4.1620e-02,
         4.2533e-02, 2.5819e-07, 1.8866e-01, 4.3364e-04, 2.5882e-02, 1.8244e-05,
         1.9992e-02, 7.4280e-05, 3.8667e-02, 1.2146e-02, 6.7027e-02, 1.0749e-02,
         2.4374e-02, 9.5655e-02, 1.0000e+00, 9.8189e-03],
        [1.0000e+00, 6.8044e-02, 3.9577e-03, 1.7364e-04, 2.8447e-01, 1.6314e-02,
         3.4427e-02, 4.8429e-02, 1.7865e-03, 6.2025e-03, 5.5201e-02, 6.6213e-03,
         1.0268e-02, 7.8733e-04, 1.6022e-01, 1.2037e-02, 6.0439e-01, 3.0455e-01,
         2.1648e-01, 1.3445e-02, 7.9118e-01, 3.8709e-01, 2.6490e-03, 6.7831e-06,
         4.2842e-01, 2.1694e-01, 4.9911e-01, 2.9940e-02, 4.8136e-01, 9.1618e-02,
         1.0000e+00, 4.6159e-03, 1.0000e+00, 3.6457e-01, 9.3417e-02, 1.5018e-01,
         1.9728e-01, 1.9091e-01, 1.8713e-01, 1.3127e-05, 4.8235e-03, 4.0605e-02,
         1.1593e-02, 1.8455e-02, 2.0881e-01, 2.0248e-01, 5.4061e-01, 1.3157e-01,
         1.9497e-01, 6.3773e-02, 3.7772e-01, 2.5146e-02, 1.8507e-01, 1.0326e-01,
         7.9414e-02, 3.4637e-04, 1.8729e-01, 2.9455e-01, 6.3960e-02, 1.2371e-01,
         3.4769e-02, 7.2885e-02, 1.0000e+00, 2.7941e-01],
        [1.0000e+00, 1.1207e-05, 1.2613e-01, 2.5937e-01, 1.4047e-01, 3.2544e-03,
         7.5789e-02, 5.6492e-02, 7.3066e-03, 8.1073e-03, 1.2852e-01, 4.1879e-02,
         4.5188e-02, 6.8054e-02, 5.1753e-01, 8.7064e-02, 5.6726e-02, 3.5027e-02,
         1.7094e-01, 2.3028e-02, 1.0978e-01, 1.1043e-02, 2.3713e-02, 1.0842e-03,
         5.7430e-02, 2.4363e-04, 4.5456e-01, 2.0183e-04, 2.1662e-01, 3.2006e-02,
         1.0000e+00, 2.8402e-03, 1.0000e+00, 1.4188e-03, 3.2565e-01, 2.0260e-02,
         1.1738e-01, 2.4412e-03, 3.5303e-01, 1.5214e-03, 1.0058e-02, 7.8154e-03,
         1.2599e-02, 4.4467e-03, 2.2918e-01, 2.1182e-05, 9.8174e-01, 4.3046e-02,
         6.0428e-02, 1.8020e-04, 4.3688e-01, 1.1216e-02, 2.8213e-02, 1.9006e-03,
         5.9339e-02, 3.4406e-03, 3.3488e-03, 6.9066e-02, 1.1997e-01, 7.6479e-03,
         3.3481e-02, 2.6526e-01, 1.0000e+00, 7.6182e-04],
        [1.0000e+00, 4.9432e-01, 5.6209e-04, 7.0812e-02, 5.2636e-02, 6.3620e-03,
         1.1224e-01, 1.0301e-01, 1.1344e-02, 1.9656e-03, 4.8228e-02, 2.3921e-02,
         8.4617e-02, 1.1551e-01, 2.7188e-01, 2.0640e-02, 1.5646e-01, 7.2944e-01,
         4.0324e-01, 3.9926e-01, 2.5276e-01, 3.6666e-01, 2.8235e-03, 6.0578e-02,
         1.9556e-01, 1.2051e-01, 2.3072e-01, 2.2092e-01, 3.6132e-01, 2.2409e-01,
         1.0000e+00, 1.1725e-02, 5.7612e-01, 6.3854e-01, 2.2977e-01, 1.6247e-02,
         1.4694e-01, 1.9101e-02, 2.2606e-01, 8.6284e-02, 3.0215e-02, 5.4907e-02,
         2.8638e-04, 1.6367e-02, 4.3295e-01, 6.8699e-03, 1.0000e+00, 4.8964e-01,
         2.8519e-02, 1.1113e-01, 1.5511e-01, 3.7017e-03, 2.3647e-02, 3.6738e-02,
         2.0718e-02, 6.6254e-04, 2.0365e-02, 3.6400e-01, 3.2260e-02, 2.3684e-01,
         6.4861e-02, 1.0000e+00, 6.4177e-01, 2.7104e-01],
        [1.0000e+00, 1.7557e-01, 4.2217e-02, 4.2704e-03, 1.1673e-01, 1.7557e-02,
         1.2798e-01, 3.8352e-03, 1.2597e-02, 5.2610e-04, 9.1602e-02, 1.0711e-03,
         4.0270e-02, 1.3464e-02, 3.6995e-01, 1.4402e-03, 4.1620e-01, 4.6574e-01,
         1.2332e-01, 2.4801e-01, 5.6579e-01, 5.0962e-01, 2.6610e-03, 5.9537e-03,
         2.8385e-01, 2.0959e-01, 6.5711e-01, 2.6525e-01, 6.6710e-01, 3.9317e-06,
         1.0000e+00, 2.9850e-02, 6.4369e-01, 4.8022e-01, 4.6432e-02, 7.6697e-03,
         1.9033e-01, 1.4118e-01, 2.3032e-01, 3.9630e-02, 8.9090e-03, 4.5658e-02,
         1.3965e-02, 2.8683e-02, 4.4770e-01, 2.6661e-01, 1.0000e+00, 4.0415e-01,
         8.4711e-02, 9.5510e-02, 1.3046e-01, 1.2189e-04, 7.6158e-02, 3.9742e-02,
         4.9796e-02, 6.7740e-03, 9.1051e-02, 3.0049e-01, 1.8531e-01, 3.2163e-01,
         3.4895e-02, 3.4823e-01, 1.0000e+00, 4.4343e-01],
        [1.0000e+00, 8.1532e-05, 8.8780e-04, 2.8992e-02, 2.3883e-01, 2.7007e-03,
         8.8976e-02, 1.1783e-02, 1.7845e-02, 1.4657e-03, 8.7283e-02, 4.6323e-03,
         1.1462e-04, 9.5346e-03, 3.4792e-01, 8.1628e-03, 5.6204e-01, 1.8470e-01,
         4.9407e-02, 6.7095e-03, 7.2275e-01, 2.1086e-01, 2.8680e-03, 1.3161e-02,
         5.1994e-01, 8.3457e-02, 7.1824e-01, 4.7832e-02, 1.0000e+00, 3.1878e-01,
         9.9776e-01, 1.5783e-01, 1.0000e+00, 1.6023e-01, 5.2288e-02, 5.3771e-01,
         2.3030e-01, 2.1173e-01, 1.8067e-01, 3.6420e-02, 1.7540e-02, 8.7783e-03,
         1.3153e-02, 1.9329e-04, 3.7771e-01, 5.7442e-01, 7.1645e-01, 2.2633e-01,
         1.5649e-01, 5.8593e-03, 2.3210e-01, 6.4585e-02, 1.5304e-01, 1.9083e-02,
         4.9501e-02, 4.3682e-03, 1.6388e-01, 6.2678e-02, 1.2812e-01, 8.7135e-02,
         6.0177e-02, 1.9688e-03, 1.0000e+00, 3.6351e-01]])

fromcode2 = np.array([[1.0000e+00, 1.1315e-04, 3.7668e-03, 4.5234e-03, 8.8233e-01, 3.5457e-04,
         2.9369e-01, 2.0238e-01, 8.7168e-02, 1.1261e-01, 4.4020e-01, 1.6932e-03,
         1.5747e-02, 2.2733e-03, 4.2671e-01, 6.8502e-03, 2.0765e-01, 1.1491e-01,
         3.6948e-02, 3.9822e-02, 9.2219e-01, 4.0045e-01, 3.9823e-02, 1.8951e-04,
         1.4789e-01, 1.6190e-01, 1.0000e+00, 4.1966e-01, 9.7695e-01, 9.8503e-02,
         5.7288e-01, 4.5502e-02, 9.6086e-01, 2.3706e-01, 4.3831e-02, 3.2116e-01,
         8.1989e-01, 7.4213e-01, 6.4714e-01, 6.2445e-02, 1.1771e-01, 3.6183e-02,
         1.5578e-01, 4.9578e-01, 5.3865e-01, 6.1229e-01, 1.0000e+00, 5.6260e-01,
         1.2343e-01, 2.6342e-02, 1.9451e-01, 3.3107e-02, 2.5043e-01, 1.7626e-02,
         3.8458e-01, 1.1172e-01, 5.4448e-02, 1.7993e-01, 5.4379e-01, 4.7583e-01,
         4.9841e-02, 7.8727e-02, 1.0000e+00, 5.1585e-01],
        [1.0000e+00, 5.9526e-01, 5.7015e-02, 9.7343e-02, 7.3036e-02, 9.0863e-03,
         4.4537e-02, 3.9309e-02, 4.6506e-02, 2.7616e-03, 9.2375e-03, 3.6222e-03,
         3.8996e-02, 5.4656e-02, 1.5364e-01, 2.9289e-02, 4.9137e-02, 4.9520e-01,
         4.2823e-01, 1.3521e-01, 1.0480e-01, 9.6653e-02, 1.8603e-04, 5.5291e-02,
         1.6864e-01, 5.5425e-02, 5.1474e-02, 1.9223e-02, 1.1869e-01, 4.2898e-02,
         1.0000e+00, 1.4706e-02, 1.0000e+00, 5.3744e-01, 7.2025e-01, 7.0693e-03,
         1.0033e-01, 4.8959e-03, 2.0365e-01, 2.1448e-02, 1.1418e-01, 6.7063e-02,
         4.6313e-02, 3.5441e-03, 2.6229e-01, 1.1560e-04, 9.1316e-01, 1.1226e-01,
         3.6776e-02, 1.4261e-01, 5.3896e-01, 5.9185e-03, 1.5191e-02, 1.3608e-01,
         3.0715e-02, 2.6344e-03, 1.5516e-02, 5.7752e-01, 1.4587e-02, 9.4635e-02,
         9.8366e-02, 6.6954e-01, 1.0000e+00, 7.3054e-02],
        [7.6123e-01, 1.5009e-01, 9.6255e-02, 3.6867e-02, 1.0000e+00, 8.4394e-02,
         1.4911e-01, 4.4988e-01, 1.5201e-01, 1.4758e-01, 2.0024e-01, 5.7379e-03,
         1.9191e-02, 1.0540e-02, 1.0430e-01, 7.0807e-03, 3.7415e-01, 8.1130e-02,
         8.4224e-02, 2.4092e-02, 1.0000e+00, 2.4146e-01, 1.0519e-02, 3.8745e-07,
         2.4771e-01, 8.9059e-02, 5.8209e-01, 2.8066e-01, 5.8149e-01, 1.1518e-01,
         4.7518e-01, 1.6142e-01, 9.4371e-01, 5.1600e-02, 8.8432e-02, 1.0000e+00,
         1.1518e-01, 5.9596e-02, 2.0702e-02, 2.9605e-03, 9.3463e-02, 6.6097e-04,
         4.2166e-02, 2.5170e-01, 3.7524e-02, 9.2900e-02, 1.6163e-01, 2.5423e-01,
         4.7032e-01, 3.6916e-02, 1.0000e+00, 5.1679e-01, 5.2273e-01, 5.0259e-02,
         1.6242e-04, 8.0372e-02, 5.1571e-02, 1.0590e-01, 3.8533e-01, 4.3322e-01,
         1.7611e-02, 6.3506e-02, 8.1946e-01, 6.3695e-01],
        [1.0000e+00, 2.8158e-01, 5.8768e-02, 2.9614e-03, 8.2670e-03, 1.6078e-02,
         8.9953e-02, 1.5317e-01, 2.5701e-02, 4.0675e-04, 1.5496e-02, 1.0155e-02,
         5.4954e-02, 1.7622e-02, 2.8354e-02, 1.1089e-05, 3.7151e-01, 1.0000e+00,
         9.3691e-02, 3.2193e-01, 3.3401e-01, 3.1821e-01, 4.2364e-03, 4.1497e-01,
         5.0334e-01, 1.6558e-01, 8.8565e-02, 1.6788e-01, 1.1810e-01, 3.0185e-02,
         5.3044e-01, 1.4076e-02, 1.0000e+00, 9.4143e-01, 5.6188e-02, 1.3732e-02,
         8.2782e-02, 4.5156e-02, 1.0743e-01, 1.9650e-01, 1.8801e-01, 1.0351e-01,
         1.2563e-03, 2.1740e-03, 1.6238e-01, 1.0342e-01, 4.8037e-01, 4.0372e-01,
         2.4178e-01, 3.1026e-01, 2.9339e-01, 1.7138e-04, 1.5751e-01, 2.0751e-01,
         1.7259e-02, 4.9687e-06, 4.1623e-01, 9.8451e-01, 2.6211e-03, 5.7690e-01,
         9.2562e-04, 1.0000e+00, 9.1014e-01, 9.2570e-01],
        [1.0000e+00, 2.4570e-02, 1.5971e-01, 1.9539e-01, 1.1265e-01, 1.4646e-03,
         6.0456e-03, 2.5121e-03, 2.1115e-02, 3.6717e-04, 7.6898e-03, 2.0552e-03,
         3.2696e-02, 6.7452e-03, 1.5398e-01, 8.9665e-02, 5.2206e-02, 1.0663e-01,
         1.9161e-01, 1.8820e-02, 6.9545e-02, 4.2084e-03, 4.3016e-04, 2.4450e-02,
         2.1722e-01, 8.1557e-04, 5.6688e-02, 6.6929e-04, 6.5304e-02, 3.2588e-04,
         1.0000e+00, 1.2847e-02, 1.0000e+00, 9.0000e-03, 4.2804e-01, 1.4955e-02,
         3.4038e-02, 1.2805e-03, 8.2243e-02, 6.5847e-04, 4.8793e-02, 2.4676e-03,
         1.3130e-02, 3.4325e-05, 6.1378e-02, 5.0756e-05, 2.9956e-01, 2.2236e-02,
         1.0293e-01, 1.6364e-03, 8.9805e-01, 1.1502e-02, 2.6807e-02, 6.8350e-02,
         2.4305e-02, 9.8334e-03, 2.9358e-02, 1.1988e-01, 6.3982e-03, 1.6172e-02,
         3.0039e-02, 1.6556e-01, 1.0000e+00, 1.0224e-03],
        [1.0000e+00, 4.1344e-02, 2.9608e-02, 1.9602e-04, 4.9338e-01, 6.1769e-03,
         1.9145e-04, 1.4018e-01, 2.7557e-02, 4.8152e-02, 1.4323e-01, 1.9478e-03,
         2.2108e-02, 3.9619e-04, 2.4522e-01, 3.8940e-04, 3.4713e-01, 1.8866e-01,
         1.1570e-01, 2.4016e-02, 7.5107e-01, 2.5060e-01, 3.2651e-02, 2.4724e-02,
         3.5480e-01, 7.5146e-02, 7.4971e-01, 1.7475e-01, 7.2534e-01, 2.9946e-02,
         1.0000e+00, 1.1036e-01, 1.0000e+00, 5.0717e-02, 1.1383e-01, 4.3939e-01,
         1.4332e-01, 7.8584e-02, 1.0251e-01, 2.5866e-03, 2.6070e-02, 9.0274e-03,
         2.9671e-02, 7.9004e-02, 1.2916e-01, 1.0063e-01, 3.7828e-01, 2.5809e-01,
         2.5922e-01, 2.1756e-03, 6.1675e-01, 9.0564e-02, 2.3635e-01, 3.6969e-02,
         2.5801e-02, 8.4593e-03, 6.0828e-02, 1.2747e-01, 2.6767e-01, 2.3923e-01,
         3.2726e-02, 1.5864e-01, 1.0000e+00, 3.9775e-01],
        [1.0000e+00, 1.8120e-01, 1.6443e-02, 3.5057e-04, 4.0300e-02, 1.1247e-02,
         8.0871e-02, 4.0876e-02, 6.9356e-02, 7.9331e-05, 2.3835e-02, 3.1810e-04,
         2.3971e-02, 7.1911e-03, 1.4555e-01, 8.3994e-03, 4.1976e-01, 6.7499e-01,
         1.2635e-01, 1.6451e-01, 3.6212e-01, 3.1181e-01, 6.2297e-04, 1.1876e-01,
         5.4740e-01, 1.6800e-01, 2.5044e-01, 5.7111e-02, 3.3970e-01, 3.8896e-03,
         1.0000e+00, 8.5350e-03, 1.0000e+00, 6.8914e-01, 9.4026e-02, 2.9869e-02,
         1.0338e-01, 5.9188e-02, 1.3709e-01, 8.1647e-02, 2.0310e-01, 7.6590e-02,
         2.7808e-03, 9.3103e-03, 3.3219e-01, 2.4978e-01, 8.6397e-01, 2.0643e-01,
         1.2570e-01, 1.2472e-01, 2.1110e-01, 1.6459e-03, 8.0220e-02, 7.3432e-02,
         1.7353e-02, 4.0911e-04, 2.2156e-01, 3.9978e-01, 1.5654e-02, 1.6841e-01,
         2.9436e-02, 2.3392e-01, 1.0000e+00, 3.5211e-01],
        [1.0000e+00, 9.0140e-01, 1.5445e-01, 5.3136e-02, 7.3018e-01, 1.5781e-01,
         2.2362e-01, 6.0240e-01, 3.4491e-02, 4.7636e-01, 3.8632e-01, 5.0436e-03,
         6.9329e-02, 1.5877e-04, 6.2044e-01, 4.2496e-02, 1.5682e-01, 2.4072e-01,
         9.8358e-02, 1.7934e-01, 6.2218e-01, 2.3385e-01, 7.3585e-02, 6.6090e-02,
         1.9494e-01, 3.9246e-02, 9.7440e-01, 5.2485e-01, 9.1448e-01, 5.3709e-02,
         1.0000e+00, 1.9215e-01, 1.0000e+00, 1.9987e-03, 1.2230e-01, 4.7883e-01,
         8.5178e-02, 1.6793e-02, 5.0484e-02, 1.3487e-03, 9.9363e-02, 2.2763e-02,
         4.8263e-02, 1.7113e-01, 2.0186e-01, 5.0477e-02, 5.1972e-01, 5.2794e-01,
         1.6287e-01, 7.7645e-02, 4.1833e-01, 4.2022e-02, 1.1277e-01, 3.2577e-03,
         5.4900e-04, 3.5238e-02, 3.9957e-03, 1.4357e-01, 5.5160e-01, 4.5377e-01,
         4.2021e-02, 6.5480e-01, 1.0000e+00, 5.3809e-01],
        [9.2657e-01, 2.0690e-01, 2.1569e-01, 1.6556e-02, 1.0000e+00, 2.1570e-02,
         6.1987e-02, 5.5352e-01, 2.8051e-03, 7.4807e-03, 1.2312e-02, 2.2146e-03,
         1.5593e-01, 8.8599e-03, 5.2771e-02, 6.5177e-02, 6.4900e-01, 5.3740e-01,
         7.4522e-02, 2.6450e-01, 1.0000e+00, 8.3379e-01, 2.6976e-05, 3.6349e-04,
         4.4279e-01, 3.8409e-01, 2.1179e-01, 4.1814e-01, 1.0198e-01, 1.4787e-02,
         3.0698e-01, 2.9797e-02, 1.0000e+00, 6.9427e-01, 1.2246e-02, 1.1730e-01,
         2.2096e-01, 2.4191e-01, 4.6647e-02, 5.6048e-02, 4.5545e-03, 6.9441e-02,
         4.1285e-02, 2.5451e-01, 2.8778e-02, 4.2755e-02, 2.4434e-01, 8.8102e-01,
         3.3712e-01, 1.4963e-01, 3.1105e-01, 3.4681e-02, 4.3676e-01, 4.5317e-01,
         2.0129e-02, 3.8054e-02, 2.7406e-01, 6.5297e-01, 7.3410e-02, 8.2376e-01,
         3.4848e-02, 7.7394e-01, 4.0928e-01, 1.0000e+00],
        [1.0000e+00, 1.3749e-02, 2.3983e-02, 3.8033e-02, 1.6091e-01, 3.1603e-04,
         5.0703e-02, 8.9543e-03, 4.4687e-03, 7.9604e-04, 6.2350e-02, 3.5939e-03,
         2.6973e-02, 1.1137e-02, 3.0924e-01, 2.0508e-02, 1.4795e-01, 1.8486e-01,
         1.5025e-01, 4.9355e-02, 2.3359e-01, 9.6688e-02, 1.3069e-02, 2.6251e-02,
         2.3085e-01, 1.9984e-02, 3.6277e-01, 2.0853e-02, 3.2727e-01, 6.0604e-04,
         1.0000e+00, 7.3699e-03, 1.0000e+00, 7.1794e-02, 2.1682e-01, 1.0124e-01,
         1.2120e-01, 3.5637e-02, 1.9459e-01, 9.2838e-03, 6.4746e-03, 1.1901e-02,
         2.3727e-03, 3.8814e-03, 2.1404e-01, 4.4968e-02, 6.9512e-01, 1.4481e-01,
         1.0950e-01, 7.2596e-03, 4.1717e-01, 1.9808e-03, 7.3252e-02, 2.9316e-02,
         3.5419e-02, 1.5248e-03, 4.8242e-02, 1.3403e-01, 7.5325e-02, 7.2754e-02,
         4.0706e-02, 2.2177e-01, 1.0000e+00, 1.1343e-01]])

In [73]:
# define two fake collections of images
images1 = fromcode1
images1 = images1.reshape((10,8,8))
images2 = fromcode2
images2 = images2.reshape((10,8,8))
print('Prepared', images1.shape, images2.shape)

Prepared (10, 8, 8) (10, 8, 8)


In [76]:
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')

# resize images
images1 = scale_images(images1, (8,8))
images2 = scale_images(images2, (8,8))
print('Scaled', images1.shape, images2.shape)

# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)

# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Scaled (10, 8, 8) (10, 8, 8)


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(10, 8, 8), dtype=float32). Expected shape (None, None, None, 3), but input has incompatible shape (10, 8, 8)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(10, 8, 8), dtype=float32)
  • training=False
  • mask=None